In [114]:
import urllib.request
import bs4
import pandas as _pd

response = urllib.request.urlopen('https://vk.com/topic-22522055_25921627?offset=20')
soup = bs4.BeautifulSoup(response, 'html.parser')

In [115]:
nodes = soup.findAll('div', {'class': 'post_item'})

In [116]:
import datetime as _dt
import re as _re

monthRus = [
    'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'
]

vkDateRegex = _re.compile('([0-9]{1,2})\W([а-я]{3})\W([0-9]{4})\Wв\W([0-9]{1,2}):([0-9]{1,2})', _re.IGNORECASE)
props = []

In [117]:
for node in nodes:
    fromId = node.find('a', {'class': 'pi_author'}).get('href')[1:]
    
    toId = node.find('a', {'class': 'mem_link'})
    toId = toId.get('href')[1:] if toId else 'null'
    
    date = node.find('a', {'class', 'item_date'}).text
    day, month, year, hours, minutes = vkDateRegex.findall(date)[0]
    date = _dt.datetime(int(year), monthRus.index(month)+1, int(day), int(hours), int(minutes), 0)
    
    likes = node.find('b', {'class', 'v_like'})
    likes = likes.text if likes else 0
    
    msg = node.find('div', {'class': 'pi_text'}).text
    
    props.append({
        'from' : str(fromId),
        'to' : toId,
        'date' : date.strftime('%Y-%m-%d %H:%M:%S'),
        'likes' : str(likes)+'',
        'msg' : '"'+msg+'"'
    })

In [118]:
separator = ',';
keys = props[0].keys()
print(keys)

dict_keys(['from', 'to', 'date', 'likes', 'msg'])


In [121]:
from os import stat as _os_stat
from os import path as _os_path
if not _os_path.isfile('vk.csv') or _os_stat('vk.csv').st_size == 0:
    file = open('vk.csv', 'w')
    file.write(separator.join(keys)+'\n')
    file.close()

In [122]:
csv = open('vk.csv', 'a')
for prop in props:
    csv.write(separator.join(prop.values())+'\n')
csv.close()

In [110]:
_pd.read_csv('vk.csv')

ParserError: Error tokenizing data. C error: Expected 4 fields in line 81, saw 15
